In [1]:
from cbench.commands import *
from cbench import state
from plumbum.machines.paramiko_machine import ParamikoMachine
from time import sleep
import _thread

In [2]:
state.RUN_NAME = "MAX_READ_6_vs_m4.2xlarge_180"
#state.WORKLOAD = "workloads/workload_read_modify_write"
state.WORKLOAD = "workloads/workload_read"
#state.WORKLOAD = "workloads/workload_scan"

In [ ]:
print("Instances:")
list_instances()
print("Loading state from AWS..")
load_state()
print("Cluster instances: {0}".format(state.CLUSTER_INSTANCES))

In [3]:
create_instances(6, state.CLUSTER_INSTANCES, type="i2.xlarge")
create_instances(1, state.YCSB_INSTANCES, 'ycsb', type="m4.2xlarge")

2016-02-09 15:05:11,876 INFO Run MAX_READ_6_vs_m4.2xlarge_180 start of 'create_instances' args: (6, []) kwargs: {'type': 'i2.xlarge'}
2016-02-09 15:05:13,727 INFO Created instance with id: i-419d10fd private ip: 172.31.16.165 public ip: None
2016-02-09 15:05:14,403 INFO Created instance with id: i-1d9c11a1 private ip: 172.31.26.136 public ip: None
2016-02-09 15:05:15,022 INFO Created instance with id: i-393ceb84 private ip: 172.31.4.167 public ip: None
2016-02-09 15:05:15,642 INFO Created instance with id: i-759c11c9 private ip: 172.31.27.251 public ip: None
2016-02-09 15:05:16,359 INFO Created instance with id: i-fd3ceb40 private ip: 172.31.6.145 public ip: None
2016-02-09 15:05:16,957 INFO Created instance with id: i-769c11ca private ip: 172.31.17.77 public ip: None
2016-02-09 15:05:16,958 INFO Run MAX_READ_6_vs_m4.2xlarge_180 end of 'create_instances'
2016-02-09 15:05:16,960 INFO Run MAX_READ_6_vs_m4.2xlarge_180 start of 'create_instances' args: (1, [], 'ycsb') kwargs: {'type': 'm4.

In [ ]:
print(state.CLUSTER_INSTANCES)

In [4]:
#Wait for Instances to finish boot
sleep(5 * 60)

In [5]:
# create the cluster
create_cluster(state.CLUSTER_INSTANCES)

2016-02-09 15:10:17,644 INFO Run MAX_READ_6_vs_m4.2xlarge_180 start of 'create_cluster' args: (['i-419d10fd', 'i-1d9c11a1', 'i-393ceb84', 'i-759c11c9', 'i-fd3ceb40', 'i-769c11ca'],) kwargs: {}
2016-02-09 15:10:18,119 INFO Checking instance 'i-419d10fd' state: {'Name': 'running', 'Code': 16}
2016-02-09 15:10:18,120 INFO Connecting to 52.29.233.100
2016-02-09 15:10:18,256 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:10:18,591 INFO Authentication (publickey) successful!
2016-02-09 15:10:20,100 INFO [chan 1] Opened sftp connection (server version 3)
2016-02-09 15:10:20,897 INFO Going to wait for 5 seconds for startup or bootstrap to finish
2016-02-09 15:10:25,899 INFO Command returned:
2016-02-09 15:10:25,900 INFO 350aa611413ca910be81598cf334a6dadec89be7fd4dac9c448d299f3bc4d6cc

2016-02-09 15:10:26,026 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:10:26,316 INFO Authentication (publickey) successful!
2016-02-09 15:10:27,215 INFO [chan 1] Opened sf

In [6]:
prepare_benchmark(workload="workloads/workload_read", name=state.RUN_NAME)

2016-02-09 15:15:50,738 INFO Run MAX_READ_6_vs_m4.2xlarge_180 start of 'prepare_benchmark' args: () kwargs: {'workload': 'workloads/workload_read', 'name': 'MAX_READ_6_vs_m4.2xlarge_180'}
2016-02-09 15:15:50,738 INFO Creating keyspace YCSB and table..
2016-02-09 15:15:50,894 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:15:51,151 INFO Authentication (publickey) successful!
2016-02-09 15:15:51,941 INFO [chan 1] Opened sftp connection (server version 3)
2016-02-09 15:15:56,702 INFO Result: 
2016-02-09 15:15:56,753 INFO Loading YCSB data for workload 'workloads/workload_read'
2016-02-09 15:15:56,919 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:15:57,186 INFO Authentication (publickey) successful!
2016-02-09 15:17:59,294 INFO Result: YCSB Client 0.1
Command line: -db com.yahoo.ycsb.db.CassandraCQLClient -threads 40 -P workloads/workload_base -P workloads/workload_read -p hosts=172.31.16.165,172.31.26.136,172.31.4.167,172.31.27.251,172.31.6.145 -lo

In [ ]:
temp = state.CLUSTER_INSTANCES[-1:]

state.CLUSTER_INSTANCES = state.CLUSTER_INSTANCES[:-1]

In [ ]:
print(temp)

In [ ]:
state.CLUSTER_INSTANCES.append(temp[0])

In [ ]:
print(state.CLUSTER_INSTANCES)

In [ ]:
# Baseline benchmark
bench = _thread.start_new_thread(start_benchmark, (200, ["-p","maxexecutiontime=300"]))
sleep(5 * 60)
wait_for_finish()
gather_results()

In [7]:
# Max Load benchmark
for num in range(7):
    num = 170 + num * 10
    state.RUN_NAME = "MAX_READ_AGAIN_6_vs_m4.2xlarge_" + str(num)
    bench = _thread.start_new_thread(start_benchmark, (num, ["-p","maxexecutiontime=300"]))
    sleep(5 * 60)
    wait_for_finish()
    gather_results()
    # Wait for compaction etc. to finish
    sleep(1 * 60)

2016-02-09 15:21:34,203 INFO Run MAX_READ_AGAIN_6_vs_m4.2xlarge_170 start of 'start_benchmark' args: (170, ['-p', 'maxexecutiontime=300']) kwargs: {}
2016-02-09 15:21:34,203 INFO Starting run 'MAX_READ_AGAIN_6_vs_m4.2xlarge_170' with workload 'workloads/workload_read'
2016-02-09 15:21:34,520 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:21:34,807 INFO Authentication (publickey) successful!
2016-02-09 15:26:34,202 INFO Run MAX_READ_AGAIN_6_vs_m4.2xlarge_170 start of 'wait_for_finish' args: () kwargs: {}
2016-02-09 15:26:34,517 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:26:34,775 INFO Authentication (publickey) successful!
2016-02-09 15:26:35,600 INFO [chan 1] Opened sftp connection (server version 3)
2016-02-09 15:26:46,013 INFO Result: 
2016-02-09 15:26:46,066 INFO Connected (version 2.0, client OpenSSH_6.6.1p1)
2016-02-09 15:26:46,176 INFO Run MAX_READ_AGAIN_6_vs_m4.2xlarge_170 end of 'start_benchmark'
2016-02-09 15:26:46,326 INFO Authentic

In [ ]:
print(state.CLUSTER_INSTANCES)
# Create additional instance
create_instances(1, state.CLUSTER_INSTANCES, type="i2.xlarge")
print("New instance: {0}".format(state.CLUSTER_INSTANCES[-1:]))

In [ ]:
scale_cluster(state.CLUSTER_INSTANCES[-1:])

In [ ]:
sleep(5)
state.CLUSTER_INSTANCES.append(temp[0])

In [ ]:
# Scaling benchmark
state.RUN_NAME = "Scale_SCAN_6_vs_m4.2xlarge_100_QUORUM_WRITE"
bench = _thread.start_new_thread(start_benchmark, (100, []))
sleep(3 * 60)
scale_cluster(state.CLUSTER_INSTANCES[-1:])
wait_for_finish()
gather_results()

In [ ]:
print(state.SEED_IP)

In [ ]:
#Reduce benchmark
state.RUN_NAME = "Reduce_SCAN_6_vs_m4.2xlarge_100_QUORUM_WRITE"
bench = _thread.start_new_thread(start_benchmark, (100, []))
sleep(3 * 60)
remove_cassandra_instance(state.CLUSTER_INSTANCES[-1])
wait_for_finish()
gather_results()

In [ ]:
#state.RUN_NAME = "Reduce_READ_MODIFY_WRITE_4_vs_m4.2xlarge_100_2"
wait_for_finish()
#gather_results()

In [ ]:
gather_results()

In [ ]:
cleanup_logs()

In [ ]:
terminate_all()

In [18]:
#plot("Scale_SCAN_6_vs_m4.2xlarge_100_QUORUM_WRITE")
plot("MAX_READ_AGAIN_6_vs_m4.2xlarge_200", granularity=10)

"


In [ ]:
from cbench import util

for inst in state.CLUSTER_INSTANCES:
    util.docker_status(inst)